In [8]:
import pandas as pd

In [18]:
df = pd.read_csv('../data/raw_data/uk_2021.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa3 in position 15580: invalid start byte

In [16]:
df.head()

,Project name,FE Capital Investment Programme,The Francis Crick Institute (formerly UKCMRI),Green Investment Bank,HE Reform Programme,Royal Mail Sale of Shares,BIS Shared Services,SLC Transformation Programme,ICR Monetisation,Urenco Future Options,...,HMCTS Reform,Legal Aid Transformation,ISOT Programme,Yoda Project (Outsourced Services Retender Project),ESA10/BPM6 Programme (European Systems of Accounting 2010),2011 Census,Electronic Data Collection (EDC),Web Data Access (WDA) Project,Beyond 2011 Programme,Unnamed: 200
0,Department,BIS,BIS,BIS,BIS,BIS,BIS,BIS,BIS,BIS,...,MoJ,MoJ,NCA,NS&I,ONS,ONS,ONS,ONS,ONS,NaN
1,MPA RAG rating\r\n(A Delivery Confidence Asses...,Amber/Green,Amber/Green,Green,Amber/Green,Amber,Amber,Amber,Amber/Red,Data exempt under section 27 and section 43 of...,...,Amber/Red,Amber,Red,Green,Amber,Amber/Green,Amber/Red,Amber/Green,Amber/Green,NaN
2,Description / Aims \r\n,To renew and modernise the FE College estate w...,The Francis Crick Institute (formerly UKCMRI) ...,"To build a green investment bank, to accelerat...",To put higher education funding on a sustainab...,To sustain the universal postal service for th...,A shared service solution for the BIS network ...,The SLC Transformation Programme supports the ...,HM Government is carrying out a programme of ...,The Government has adopted a policy of asset d...,...,The aim of the HMCTS Reform Programme is to:\r...,The aim of the Legal Aid Transformation Progra...,The SOCA ICT MODERNISATION Programme (previous...,NS&I provides the Government with access to re...,Meeting current and new international legal re...,"Design, build and deliver the 2011 Census and ...","Developing systems, methods and processes to ...",Building on the ONS website and delivering new...,Current phase - To recommend the best way to p...,NaN
3,Departmental commentary on actions planned or ...,MPA RAG rating as at 30 September 2013 was Amb...,The Amber/Green status reflects the good progr...,GIB became fully operational in October 2012 f...,The Amber/Green status reflects that student n...,The Amber status reflects the view that the pr...,The Amber status reflects the challenges face...,The Amber status reflected the requirement to ...,Actions being taken to address the Amber/Red r...,Data exempt under section 27 and section 43 of...,...,The Amber/Red rating was based on the complexi...,The Programme has an Amber rating. This reflec...,To address the RAG status at Q2 13/14 the Prog...,The project team have no outstanding correctiv...,ONS/MPA narrative at Q2 2013:\r\n\r\nThe progr...,ONS/MPA narrative at Q2 2013:\r\n\r\nCensus 20...,ONS/MPA narrative at Q2 2013:\r\n\r\nThe Elect...,ONS/MPA narrative at Q2 2013:\r\n\r\nThe Web D...,ONS/MPA narrative at Q2 2013:\r\n\r\nBeyond 20...,NaN
4,Project - Start Date\r\n(Latest approved start...,31/05/2010,01/10/2006,01/06/2010,30/06/2011,01/04/2012,04/05/2011,01/03/2012,01/03/2010,01/02/2010,...,21/03/2013,04/12/2012,01/04/2008,05/04/2009,05/02/2011,01/09/2002,02/04/2012,01/04/2011,01/04/2011,NaN
